In [149]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import os.path
from os import listdir
import multiprocessing
from tqdm import tqdm
import mmap
from multiprocessing import Pool
import seaborn as sns
import matplotlib.pyplot as plt
import py_entitymatching as em
import py_stringmatching as sm
import scipy
from scipy.io import arff
from sklearn import preprocessing
#from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split

In [80]:
#read table to dataframe and create train/test split with equal distribution of labels within table
#example small book tables
small = pd.read_csv('../../src/data/schemafiltereddata/small_tables.csv')
small_books = pd.DataFrame()
small_books = small[small['Class']=='Book']
#pd.set_option('display.max_columns', None)
small_books

,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,...,Restaurant_menu,Restaurant_name,Restaurant_openinghours,Restaurant_openinghoursspecification,Restaurant_pricerange,Restaurant_servescuisine,Restaurant_telephone,TVEpisode_episodenumber,TVEpisode_name,TVEpisode_partofseries
0,Book_aardvark-books.com_September2020.json.gz,Book,26.0,9.0,7.0,5,0.115385,0.027473,35.0,0,...,0,0,0,0,0,0,0,0,0,0
1,Book_americanreadingathome.com_September2020.json.gz,Book,33.0,9.0,5.0,3,0.090909,0.018182,15.0,0,...,0,0,0,0,0,0,0,0,0,0
2,Book_barmatrioshka.com_September2020.json.gz,Book,35.0,11.0,9.0,0,0.000000,0.000000,0.0,1,...,0,0,0,0,0,0,0,0,0,0
3,Book_bibliobeat.com_September2020.json.gz,Book,22.0,10.0,6.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0,0,0
4,Book_bookapy.com_September2020.json.gz,Book,26.0,13.0,8.0,11,0.307692,0.052885,88.0,0,...,0,0,0,0,0,0,0,0,0,0
5,Book_books.org_September2020.json.gz,Book,36.0,8.0,5.0,11,0.305556,0.061111,55.0,0,...,0,0,0,0,0,0,0,0,0,0
6,Book_capitalsoftworks.co.uk_September2020.json.gz,Book,25.0,13.0,7.0,17,0.280000,0.097143,119.0,0,...,0,0,0,0,0,0,0,0,0,0
7,Book_carlosruizzafon.co.uk_September2020.json.gz,Book,27.0,10.0,5.0,1,0.037037,0.007407,5.0,0,...,0,0,0,0,0,0,0,0,0,0
8,Book_clairenorth.com_September2020.json.gz,Book,22.0,11.0,6.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0,0,0
9,Book_cressidacowell.co.uk_September2020.json.gz,Book,21.0,11.0,6.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
small_books['id'] = small_books.index
#small_books

In [16]:
book_col = list([col for col in small_books if col.startswith('Book_')])
book_col

['Book_aggregaterating',
 'Book_author',
 'Book_bookedition',
 'Book_bookformat',
 'Book_datepublished',
 'Book_genre',
 'Book_inlanguage',
 'Book_isbn',
 'Book_name',
 'Book_numberofpages',
 'Book_offers',
 'Book_publisher']

In [17]:
X = small_books
train_size = int(len(X) * 0.7)
train, test = X[0:train_size], X[train_size:len(X)]
#X_train, y_train, X_test, y_test = iterative_train_test_split(train,test, test_size = 0.2)
#test = test.values
print('Total Counts: %d' % (len(X)))
print('Train Counts: %d' % (len(train)))
print('Testing Counts: %d' % (len(test)))

Total Counts: 43
Train Counts: 30
Testing Counts: 13


In [54]:
X = small_books['filename']
y = small_books[['Book_aggregaterating','Book_author','Book_bookedition','Book_bookformat','Book_datepublished','Book_genre','Book_inlanguage','Book_isbn','Book_name','Book_numberofpages','Book_offers','Book_publisher']]
t_train, y_train, t_test, y_test = train_test_split(X, y, test_size = 0.5)
t_train

41               Book_worklizard.com_September2020.json.gz
26            Book_pspmrsmtumpat.com_September2020.json.gz
11               Book_double-eye.com_September2020.json.gz
16            Book_jillmansell.co.uk_September2020.json.gz
19               Book_marasworld.com_September2020.json.gz
7         Book_carlosruizzafon.co.uk_September2020.json.gz
3                Book_bibliobeat.com_September2020.json.gz
20           Book_memopublishers.com_September2020.json.gz
0            Book_aardvark-books.com_September2020.json.gz
27                 Book_publicvm.com_September2020.json.gz
21                   Book_mrface.com_September2020.json.gz
28                     Book_qpoe.com_September2020.json.gz
33         Book_stepheniemeyer.co.uk_September2020.json.gz
14               Book_hologram.me.uk_September2020.json.gz
4                   Book_bookapy.com_September2020.json.gz
17      Book_librairie-anagramme.com_September2020.json.gz
15               Book_jamesjaffe.com_September2020.json.

In [ ]:
#from skmultilearn.model_selection import iterative_train_test_split
#X = small_books['index']
#y = small_books[['Book_aggregaterating','Book_author','Book_bookedition','Book_bookformat','Book_datepublished','Book_genre','Book_inlanguage','Book_isbn','Book_name','Book_numberofpages','Book_offers','Book_publisher']]
#t_train, y_train, t_test, y_test = iterative_train_test_split(X, y, test_size = 0.2)

In [76]:
#stratified split to make sure that all selected columns/labels are equally distributed in tables used for training and testing
#test size = 0.2

from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
import numpy as np

X = np.array(small_books['filename'])
y = np.array(small_books[['Book_aggregaterating','Book_author','Book_bookedition','Book_bookformat','Book_datepublished','Book_genre','Book_inlanguage','Book_isbn','Book_name','Book_numberofpages','Book_offers','Book_publisher']])

msss = MultilabelStratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=0)

for train_index, test_index in msss.split(X, y):
   print("TRAIN:", train_index, "TEST:", test_index)
   X_train, X_test = X[train_index], X[test_index]
   y_train, y_test = y[train_index], y[test_index]



TRAIN: [ 1  2  3  4  5  6  8  9 10 11 12 13 14 15 17 18 21 22 24 25 27 28 29 30
 31 32 33 35 36 38 39 42] TEST: [ 0  7 16 19 20 23 26 34 37 40 41]
TRAIN: [ 0  1  2  3  6  7  8  9 10 12 13 14 15 16 17 18 19 21 22 24 25 26 28 29
 30 31 32 35 36 38 40 41 42] TEST: [ 4  5 11 20 23 27 33 34 37 39]
TRAIN: [ 1  2  3  4  5  6  7  8  9 10 11 14 16 17 18 19 20 21 22 24 25 26 27 28
 29 31 32 33 34 37 39 40 41 42] TEST: [ 0 12 13 15 23 30 35 36 38]


In [102]:
#test_index
train_df = small_books[small_books['id'].isin(train_index)]
test_df = small_books[small_books['id'].isin(test_index)]
print(test_df['filename'])
print(train_df['filename'])

0        Book_aardvark-books.com_September2020.json.gz
12     Book_george-pelecanos.com_September2020.json.gz
13        Book_gladwellbooks.com_September2020.json.gz
15           Book_jamesjaffe.com_September2020.json.gz
23              Book_onedumb.com_September2020.json.gz
30              Book_rexcity.com_September2020.json.gz
35              Book_suninme.org_September2020.json.gz
36    Book_takethedayoffbook.com_September2020.json.gz
38             Book_toddparr.com_September2020.json.gz
Name: filename, dtype: object
1         Book_americanreadingathome.com_September2020.json.gz
2                 Book_barmatrioshka.com_September2020.json.gz
3                    Book_bibliobeat.com_September2020.json.gz
4                       Book_bookapy.com_September2020.json.gz
5                         Book_books.org_September2020.json.gz
6            Book_capitalsoftworks.co.uk_September2020.json.gz
7             Book_carlosruizzafon.co.uk_September2020.json.gz
8                   Book_clairenor

In [91]:
train_agg = train_df.groupby('Class').agg({'Book_aggregaterating':'sum','Book_author':'sum','Book_bookedition':'sum','Book_bookformat':'sum','Book_datepublished':'sum', 'Book_genre':'sum','Book_inlanguage':'sum','Book_isbn':'sum','Book_name':'sum','Book_numberofpages':'sum','Book_offers':'sum','Book_publisher':'sum'})
train_agg #small_books.groupby('Class').agg({'Book_aggregaterating':'sum','Book_author':'sum','Book_bookedition':'sum'})

,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,Book_name,Book_numberofpages,Book_offers,Book_publisher
Class,,,,,,,,,,,,
Book,6,29,3,15,25,13,9,29,34,22,9,19


In [92]:
test_agg = test_df.groupby('Class').agg({'Book_aggregaterating':'sum','Book_author':'sum','Book_bookedition':'sum','Book_bookformat':'sum','Book_datepublished':'sum', 'Book_genre':'sum','Book_inlanguage':'sum','Book_isbn':'sum','Book_name':'sum','Book_numberofpages':'sum','Book_offers':'sum','Book_publisher':'sum'})
test_agg #

,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,Book_name,Book_numberofpages,Book_offers,Book_publisher
Class,,,,,,,,,,,,
Book,2,8,1,4,7,4,2,8,9,6,3,7


In [73]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import numpy as np

X = np.array(small_books['filename'])
y = np.array(small_books[['Book_aggregaterating','Book_author','Book_bookedition','Book_bookformat','Book_datepublished','Book_genre','Book_inlanguage','Book_isbn','Book_name','Book_numberofpages','Book_offers','Book_publisher']])

mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=0)

for train_index, test_index in mskf.split(X, y):
   print("TRAIN:", train_index, "TEST:", test_index)
   X_train, X_test = X[train_index], X[test_index]
   y_train, y_test = y[train_index], y[test_index]

TRAIN: [ 0  1  2  3  4  5  7  8  9 10 12 13 15 16 18 19 21 23 24 25 26 27 28 29
 30 31 33 34 35 36 37 38 39 40 42] TEST: [ 6 11 14 17 20 22 32 41]
TRAIN: [ 0  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 20 21 22 23 24 27 28 30
 31 32 33 34 35 36 37 38 39 41 42] TEST: [ 1  2 10 19 25 26 29 40]
TRAIN: [ 0  1  2  3  4  6  7  8  9 10 11 12 13 14 15 17 18 19 20 21 22 24 25 26
 27 28 29 31 32 33 34 38 40 41] TEST: [ 5 16 23 30 35 36 37 39 42]
TRAIN: [ 0  1  2  4  5  6  7 10 11 12 14 16 17 19 20 22 23 24 25 26 27 29 30 31
 32 33 35 36 37 38 39 40 41 42] TEST: [ 3  8  9 13 15 18 21 28 34]
TRAIN: [ 1  2  3  5  6  8  9 10 11 13 14 15 16 17 18 19 20 21 22 23 25 26 28 29
 30 32 34 35 36 37 39 40 41 42] TEST: [ 0  4  7 12 24 27 31 33 38]


In [22]:
df_grouped = small_books.groupby('Class').agg({'Book_aggregaterating':'sum','Book_author':'sum','Book_bookedition':'sum','Book_bookformat':'sum','Book_datepublished':'sum', 'Book_genre':'sum','Book_inlanguage':'sum','Book_isbn':'sum','Book_name':'sum','Book_numberofpages':'sum','Book_offers':'sum','Book_publisher':'sum'})
df_grouped #small_books.groupby('Class').agg({'Book_aggregaterating':'sum','Book_author':'sum','Book_bookedition':'sum'})

,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,Book_name,Book_numberofpages,Book_offers,Book_publisher
Class,,,,,,,,,,,,
Book,8,37,4,19,32,17,11,37,43,28,12,26


In [13]:
small_counts = small.groupby('Class').agg(lambda x : x.sum() if x.dtype=='int64' else x.head(1))
small_counts = small_counts.reset_index()
small_counts.drop(small_counts.iloc[:, 1:9], inplace = True, axis = 1)
#small_counts.to_csv('../../src/data/data/CSV_files/small_coloum_count_overiew_class.csv')
small_counts

,Class,Book_aggregaterating,Book_author,Book_bookedition,Book_bookformat,Book_datepublished,Book_genre,Book_inlanguage,Book_isbn,Book_name,...,Restaurant_menu,Restaurant_name,Restaurant_openinghours,Restaurant_openinghoursspecification,Restaurant_pricerange,Restaurant_servescuisine,Restaurant_telephone,TVEpisode_episodenumber,TVEpisode_name,TVEpisode_partofseries
0,Book,8,37,4,19,32,17,11,37,43,...,0,0,0,0,0,0,0,0,0,0
1,CreativeWork,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Event,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hotel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,LocalBusiness,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,MusicAlbum,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,MusicRecording,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Person,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Place,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Product,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#from skmultilearn.model_selection import iterative_train_test_split
#X = small_books['filename']
#y = small_books.columns[small_books.columns.isin(book_col)]
#t_train, y_train, t_test, y_test = iterative_train_test_split(X, y, test_size = 0.2)

In [123]:
#get all columns (relations) which are represented in less than 20 tables
small_table_count = pd.read_csv('../../src/data/data/CSV_files/final_column_table_count.csv')
small_table_count = small_table_count[small_table_count['Count']<20]
small_table_count

,Unnamed: 0,Column,Count
63,63,LocalBusiness_founder,17.0
71,71,LocalBusiness_owns,2.0
106,106,Person_mainentityofpage,16.0
147,147,Product_itemlistelement,18.0
154,154,Product_offerdetails,18.0


In [127]:
#drop all columns (relations) which are represented in less than 20 tables
relevant = pd.read_csv('../../src/data/data/CSV_files/relevant_tables.csv')
small = pd.read_csv('../../src/data/schemafiltereddata/small_tables.csv')
medium = pd.read_csv('../../src/data/schemafiltereddata/medium_tables.csv')
large = pd.read_csv('../../src/data/schemafiltereddata/large_tables.csv')

In [128]:
#drop all columns (relations) which are represented in less than 20 tables
relevant = relevant.drop(['LocalBusiness_founder', 'LocalBusiness_owns','Person_mainentityofpage', 'Product_itemlistelement', 'Product_offerdetails' ], axis=1)
small = small.drop(['LocalBusiness_founder', 'LocalBusiness_owns','Person_mainentityofpage', 'Product_itemlistelement', 'Product_offerdetails' ], axis=1)
medium = medium.drop(['LocalBusiness_founder', 'LocalBusiness_owns','Person_mainentityofpage', 'Product_itemlistelement', 'Product_offerdetails' ], axis=1)
large = large.drop(['LocalBusiness_founder', 'LocalBusiness_owns','Person_mainentityofpage', 'Product_itemlistelement', 'Product_offerdetails' ], axis=1)


In [131]:
relevant.to_csv('../../src/data/data/CSV_files/relevant_tables_more_than_20.csv')
small.to_csv('../../src/data/schemafiltereddata/small_tables_more_than_20.csv')
medium.to_csv('../../src/data/schemafiltereddata/medium_tables_more_than_20.csv')
large.to_csv('../../src/data/schemafiltereddata/large_tables_more_than_20.csv')

In [156]:
#move book train/test tables to folder src/data/schemafiltereddata/TrainTestTables
#create set of tables in test and train
train_set = set(small_books.filename[small_books['id'].isin(train_index)])
test_set = set(small_books.filename[small_books['id'].isin(test_index)])
inputdir = '../../src/data/schemafiltereddata/'+'Book/'
outputdir = '../../src/data/schemafiltereddata/TrainTestTables/Small/'
files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
for file in files:
    # read file
    #df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
    for file in train_set:
        print(file)
        df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
        df.to_json(os.path.join(outputdir+'Train/'+file), compression='gzip', orient='records', lines=True)
    for file in test_set:
        print(file)
        df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
        df.to_json(os.path.join(outputdir+'Test/'+file), compression='gzip', orient='records', lines=True)
    
    
        

Book_qpoe.com_September2020.json.gz
Book_liversite.co.uk_September2020.json.gz
Book_mrface.com_September2020.json.gz
Book_worklizard.com_September2020.json.gz
Book_sheilaoflanagan.com_September2020.json.gz
Book_officecourseware.co.uk_September2020.json.gz
Book_barmatrioshka.com_September2020.json.gz
Book_roadtripusa.com_September2020.json.gz
Book_portablepress.com_September2020.json.gz
Book_whiterabbitbooks.co.uk_September2020.json.gz
Book_studiofun.com_September2020.json.gz
Book_books.org_September2020.json.gz
Book_stepheniemeyer.co.uk_September2020.json.gz
Book_americanreadingathome.com_September2020.json.gz
Book_pdfcloudbook.com_September2020.json.gz
Book_librairie-anagramme.com_September2020.json.gz
Book_capitalsoftworks.co.uk_September2020.json.gz
Book_memopublishers.com_September2020.json.gz
Book_tom-holland.org_September2020.json.gz
Book_davidsedarisbooks.com_September2020.json.gz
Book_jillmansell.co.uk_September2020.json.gz
Book_bibliobeat.com_September2020.json.gz
Book_publicv

In [153]:
#function to get and move selected train & test tables
def move(tables):
    #for file in files:
    # read file
    #df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
    for file in train_set:
        print(file)
        df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
        df.to_json(os.path.join(outputdir+'Train/'+file), compression='gzip', orient='records', lines=True)
    for file in test_set:
        print(file)
        df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
        df.to_json(os.path.join(outputdir+'Test/'+file), compression='gzip', orient='records', lines=True)

In [154]:
if __name__ == '__main__':
    for index in listcolt.filename:
        print(index)
        setcol = set(listcolt.loc[index]['listofcolumns'])
        print(setcol)
        inputdir = '../../src/data/schemafiltereddata/'+index+'/'
        outputdir = '../../src/data/schemafiltereddata/TrainTesttables/Small/'
        train_set = set(small.filename[small['id'].isin(train_index)])
        test_set = set(small.filename[small['id'].isin(test_index)])
        files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
        p = Pool(100)
        #for _ in tqdm(p.imap_unordered(move, [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]), total=len([file for file in os.listdir(inputdir) if file.endswith('.json.gz')])):
        #    pass
        p.map(move, [file for file in os.listdir(inputdir) if file.endswith('.json.gz')])
        p.close()
        #print(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
        p.join()
        

UnboundLocalError: local variable 'file' referenced before assignment

In [140]:
train_set = set(small_books.filename[small_books['id'].isin(train_index)])
test_set = set(small_books.filename[small_books['id'].isin(test_index)])
test_set

{'Book_aardvark-books.com_September2020.json.gz',
 'Book_george-pelecanos.com_September2020.json.gz',
 'Book_gladwellbooks.com_September2020.json.gz',
 'Book_jamesjaffe.com_September2020.json.gz',
 'Book_onedumb.com_September2020.json.gz',
 'Book_rexcity.com_September2020.json.gz',
 'Book_suninme.org_September2020.json.gz',
 'Book_takethedayoffbook.com_September2020.json.gz',
 'Book_toddparr.com_September2020.json.gz'}